In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import spacy
from transformers import MarianMTModel, MarianTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from spacy import displacy
import nltk
from summarizer import Summarizer


C:\Users\Mohamed ROS\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
summarizer = Summarizer()

In [3]:
nlp_en = spacy.load("en_core_web_sm")
nlp_ar = spacy.load("xx_ent_wiki_sm")

In [4]:
model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


In [5]:

# Function to perform sentiment analysis
def perform_sentiment_analysis(document_text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(document_text)
    if sentiment_scores['compound'] >= 0.05:
        return 'Positive'
    elif sentiment_scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [6]:
# Function to perform named entity recognition
def perform_ner(document_text):
    doc = nlp_en(document_text)
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

In [7]:
def perform_text_summarization(document_text, language):
    if language == 'en':
        summary = summarizer(document_text)
    else:
        translated_text = translate_to_arabic(document_text)
        summary = summarizer(translated_text)
    return summary

In [8]:
def translate_to_arabic(english_text):
    input_ids = tokenizer.encode(english_text, return_tensors="pt")
    translated_ids = model.generate(input_ids)
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translated_text



In [9]:
# Function to open and process the document
def open_document():
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    if file_path:
        with open(file_path, 'r') as file:
            document_text = file.read()
        # Perform NLP tasks
        sentiment = perform_sentiment_analysis(document_text)
        entities = perform_ner(document_text)
        english_summary = perform_text_summarization(document_text, 'en')
        arabic_summary = perform_text_summarization(document_text, 'ar')

        # Display results
        messagebox.showinfo("Sentiment", f"Sentiment: {sentiment}")
        messagebox.showinfo("Named Entities", f"Named Entities: {entities}")
        messagebox.showinfo("English Summary", f"English Summary:\n{english_summary}")
        messagebox.showinfo("Arabic Summary", f"Arabic Summary:\n{arabic_summary}")


In [10]:
# Create GUI
nltk.download('vader_lexicon')

root = tk.Tk()
root.title("Document Analysis App")

# Create and pack widgets
open_button = tk.Button(root, text="Open Document", command=open_document)
open_button.pack(pady=20)

root.mainloop()

[nltk_data] Downloading package vader_lexicon to C:\Users\Mohamed
[nltk_data]     ROS\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
